# Term Paper

### Part 1

In [12]:
import pandas as pd
import glob as glob

In [ ]:
files=glob.glob(f'data/SCE-*-20*.csv')

df_list=[]

waves_number=0

#Loop to read in and append files
for file in files:
    df=pd.read_csv(file,sep=";",parse_dates=['date'])
    df_list.append(df)


sce_df=pd.concat(df_list,ignore_index=True)

# Evaluating unique individuals inside the dataframe 
unique_individuals = sce_df['userid'].nunique() 

# Userid column contains unique identifiers for individuals
print(f"Number of unique individuals: {unique_individuals}")

# Number of rows in the DataFrame 
num_rows = sce_df.shape[0]
print(f"Number of rows in the DataFrame: {num_rows}")

# Setting date as index and sorting
sce_df.set_index('date',inplace=True)
sce_df.sort_index(inplace=True)



Number of unique individuals: 23369
Number of rows in the DataFrame: 176101


,userid,wid,weight,female,educ,age,hispanic,black,couple,num_kids,...,num_lit_q3,num_lit_q3_correct,num_lit_q5,num_lit_q5_correct,num_lit_q6,num_lit_q6_correct,num_lit_q8,num_lit_q8_correct,num_lit_q9,num_lit_q9_correct
date,,,,,,,,,,,,,,,,,,,,,
2013-06-01,70027661,201306,2.2,1.0,2.0,54.0,0.0,0.0,1.0,1.0,...,10.0,1.0,100.0,1.0,5.0,1.0,NaN,NaN,NaN,NaN
2013-06-01,70027730,201306,0.3,0.0,3.0,68.0,0.0,0.0,0.0,1.0,...,100.0,0.0,100.0,1.0,5.0,1.0,NaN,NaN,NaN,NaN
2013-06-02,70027707,201306,0.3,1.0,4.0,42.0,0.0,0.0,0.0,2.0,...,10.0,1.0,100.0,1.0,2.0,0.0,NaN,NaN,NaN,NaN
2013-06-02,70011306,201306,1.4,1.0,4.0,37.0,0.0,0.0,1.0,2.0,...,10.0,1.0,10.0,0.0,5.0,1.0,NaN,NaN,NaN,NaN
2013-06-02,70011246,201306,4.2,1.0,3.0,37.0,1.0,1.0,1.0,2.0,...,2.0,0.0,0.0,0.0,5.0,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31,75023024,202412,0.6,1.0,4.0,46.0,0.0,1.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-12-31,75019318,202412,0.8,0.0,4.0,58.0,0.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-12-31,75022264,202412,1.3,0.0,3.0,63.0,1.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
